# Conneting with SurrealDB

In [2]:
from surrealdb import Surreal

async def connect_to_db():
    # Connect to SurrealDB running on localhost
    db = Surreal("ws://localhost:8000/rpc")
    # db ="ws://localhost:8000/rpc"
    await db.connect()
    # Use your namespace and database
    await db.signin({"user": "root", "pass": "root"})  # Default credentials
    await db.use("test_namespace", "test_database")
    return db

# Connect and check the connection
db = await connect_to_db()
print("Connected to SurrealDB")


Connected to SurrealDB


# Installing Typesense

In [ ]:
pip install typesense

In [6]:
import typesense

client = typesense.Client({
  'nodes': [{
    'host': 'z6jqcbve7pn3m9r1p-1.a1.typesense.net', # For Typesense Cloud use xxx.a1.typesense.net
    'port': '443',      # For Typesense Cloud use 443
    'protocol': 'https'   # For Typesense Cloud use https
  }],
  'api_key': 'Q96xfhwdlvRfyAaPqfQee7INNaqCh56c',
  'connection_timeout_seconds': 2
})


# Connecting Surrealdb with typesense

In [8]:
# Fetch products from SurrealDB
async def fetch_products_from_surrealdb(db):
    try:
        products = await db.query("SELECT * FROM product;")
        if products:
            return products[0]['result']  # Extract the actual products from query result
        else:
            raise ValueError("No products found in SurrealDB.")
    except Exception as e:
        print(f"Error fetching products from SurrealDB: {e}")
        return []



In [18]:
# Index data into Typesense
def index_products_to_typesense(products):
    try:
        documents = [
            {
                "id": product['id'],
                "name": product['name'],
                "category": product['category'],
                "price": product['price'],
                "description": product['description']
            }
            for product in products
        ]

        # Index data into Typesense
        client.collections['products'].documents.import_(documents, {'action': 'upsert'})
        print("Products indexed successfully in Typesense.")
    except Exception as e:
        print(f"Error indexing products to Typesense: {e}")



In [16]:
# Main function to fetch data from SurrealDB and index to Typesense
async def sync_surrealdb_to_typesense(db):
    products = await fetch_products_from_surrealdb(db)
    if products:
        index_products_to_typesense(products)

# Call the sync function
await sync_surrealdb_to_typesense(db)

Products indexed successfully in Typesense.


# searching for required products through typesense
## make sure all products in surrealdb are correctly synced with typesense

In [20]:
# Define search parameters to find products similar to 'Gaming Headset'
search_parameters = {
  'q'        : 'Gaming',
  'query_by' : 'name, description',
  'filter_by': 'category:=Gaming',
  'sort_by'  : 'price:asc'
}

# Perform the search
search_results = client.collections['products'].documents.search(search_parameters)
print(search_results)


{'facet_counts': [], 'found': 6, 'hits': [{'document': {'category': 'Gaming', 'description': 'High precision gaming mouse with customizable RGB lighting.', 'id': 'product:product_014', 'name': 'Gaming Mouse', 'price': 59.99}, 'highlight': {'description': {'matched_tokens': ['gaming'], 'snippet': 'High precision <mark>gaming</mark> mouse with customizable RGB lighting.'}, 'name': {'matched_tokens': ['Gaming'], 'snippet': '<mark>Gaming</mark> Mouse'}}, 'highlights': [{'field': 'name', 'matched_tokens': ['Gaming'], 'snippet': '<mark>Gaming</mark> Mouse'}, {'field': 'description', 'matched_tokens': ['gaming'], 'snippet': 'High precision <mark>gaming</mark> mouse with customizable RGB lighting.'}], 'text_match': 578730123365187706, 'text_match_info': {'best_field_score': '1108091338752', 'best_field_weight': 15, 'fields_matched': 2, 'num_tokens_dropped': 0, 'score': '578730123365187706', 'tokens_matched': 1, 'typo_prefix_score': 0}}, {'document': {'category': 'Gaming', 'description': 'High-

In [22]:
# Define search parameters to find products in the Electronics category
search_parameters = {
  'q'        : 'Office',       # Search term
  'query_by' : 'category',          # Search within the 'category' field
  'filter_by': 'category:=Office',  # Filter to only include products in the Electronics category
  'sort_by'  : 'price:desc'         # Sort by price, descending (most expensive first)
}

# Perform the search in Typesense
search_results = client.collections['products'].documents.search(search_parameters)
print(search_results)


{'facet_counts': [], 'found': 3, 'hits': [{'document': {'category': 'Office', 'description': 'Adjustable standing desk to improve posture and reduce fatigue.', 'id': 'product:product_021', 'name': 'Standing Desk', 'price': 299.99}, 'highlight': {'category': {'matched_tokens': ['Office'], 'snippet': '<mark>Office</mark>'}}, 'highlights': [{'field': 'category', 'matched_tokens': ['Office'], 'snippet': '<mark>Office</mark>'}], 'text_match': 578730123365711993, 'text_match_info': {'best_field_score': '1108091339008', 'best_field_weight': 15, 'fields_matched': 1, 'num_tokens_dropped': 0, 'score': '578730123365711993', 'tokens_matched': 1, 'typo_prefix_score': 0}}, {'document': {'category': 'Office', 'description': 'Compact wireless printer for home or office use.', 'id': 'product:product_020', 'name': 'Wireless Printer', 'price': 149.99}, 'highlight': {'category': {'matched_tokens': ['Office'], 'snippet': '<mark>Office</mark>'}}, 'highlights': [{'field': 'category', 'matched_tokens': ['Offi

In [34]:
# Define search parameters with filter and sort
search_parameters = {
    'q'         : 'Electronics',                # Search term
    'query_by'  : 'category',          # Fields to search within
    'filter_by' : 'category:=Electronics',  # Filters
    'sort_by'   : 'price:asc'                   # Sorting
}

# Perform the search
search_results = client.collections['products'].documents.search(search_parameters)
print(search_results)


{'facet_counts': [], 'found': 6, 'hits': [{'document': {'category': 'Electronics', 'description': '5000mAh portable charger with dual USB output.', 'id': 'product:product_007', 'name': 'Portable Charger', 'price': 25.99}, 'highlight': {'category': {'matched_tokens': ['Electronics'], 'snippet': '<mark>Electronics</mark>'}}, 'highlights': [{'field': 'category', 'matched_tokens': ['Electronics'], 'snippet': '<mark>Electronics</mark>'}], 'text_match': 578730123365711993, 'text_match_info': {'best_field_score': '1108091339008', 'best_field_weight': 15, 'fields_matched': 1, 'num_tokens_dropped': 0, 'score': '578730123365711993', 'tokens_matched': 1, 'typo_prefix_score': 0}}, {'document': {'category': 'Electronics', 'description': 'Multi-port USB-C hub with HDMI and USB 3.0.', 'id': 'product:product_004', 'name': 'USB-C Hub', 'price': 34.99}, 'highlight': {'category': {'matched_tokens': ['Electronics'], 'snippet': '<mark>Electronics</mark>'}}, 'highlights': [{'field': 'category', 'matched_tok